In [ ]:
import os
import sys
from time import sleep
from uuid import uuid4

import GCode
import GRBL
import numpy as np

# Laser Circles 

# Code:

In [22]:
cnc = GRBL.GRBL(port="/dev/cnc_3018")
sleep(0.5)
cnc.laser_mode = 1
sleep(0.5)
print("Laser Mode: {}".format(cnc.laser_mode))
cnc.home()


ok
Laser Mode: 1.0


10

In [23]:
def init():
    program = GCode.GCode()
    program.G0(F=500)
    program.G1(F=300)
    program.M3(S=1)
    program.G28()
    program.G21()  # Metric Units
    program.G90()  # Absolute positioning.
    program.G92(X=0, Y=0)
    program.M5()
    return program


def end():
    program = GCode.GCode()
    program.M5()
    program.G28()
    return program


In [24]:
def circle(center_x=5, center_y=5, radius=5, laser_pwm=255):
    prog = GCode.GCode()
    prog.G0(X=center_x - radius, Y=center_y - radius)
    prog.M3(S=laser_pwm)
    prog.G2(X=center_x - radius, Y=center_y - radius, I=radius, J=radius)
    prog.M5()
    return prog


In [11]:
circle()


<GCode>[cmds=4]

In [26]:
radius = 10


In [27]:
# Poplar 1x4". Cut
BlockHeight = 89.0  # mm
BlockLength = 3 * BlockHeight  # mm


In [28]:
Y_centers = np.linspace(2 * radius, BlockHeight - 2 * radius, 4)
X_centers = np.linspace(2 * radius, BlockLength - 2 * radius, 5)


In [29]:
X_centers


array([ 20.  ,  76.75, 133.5 , 190.25, 247.  ])

In [30]:
test_run = GCode.GCode()
test_run += init()
for x_center in X_centers:
    for y_center in Y_centers:
        test_run += circle(
            center_x=x_center, center_y=y_center, radius=radius, laser_pwm=100
        )
test_run += end()


In [31]:
gcode_file = "CircleTests.gcode"


In [32]:
test_run.save(gcode_file)

del test_run
test_run = GCode.GCode()

test_run.load(gcode_file)


In [33]:
test_run.machine = cnc


In [34]:
test_run.run()


In [20]:
try:
    cnc.run(test_run)
    while 1:
        print(cnc.status)
        sleep(5)
except KeyboardInterrupt as error:
    print("Feed Hold")
    cnc.cmd("!")
    while 1:
        try:
            cnc.reset()
            break
        except:
            pass
    print("^C")


SerialException: read failed: device reports readiness to read but returned no data (device disconnected or multiple access on port?)

In [ ]:
cnc.cmd("G0X0Y0")


In [ ]:
cnc.reset()


In [ ]:
cnc.reset()


In [ ]:
picture()


# Test Aborted.

Cuts were way too aggressive.